---

title: Linux GPIO 的使用
keywords: maixpy3, GPIO
desc: maixpy3 doc: GPIO

---

| 更新时间 | 负责人 | 内容 | 备注 |
| --- | --- | --- | --- |
| 2022年1月4日 | Rui | 初次编写文档 | ---- |
| 2022年1月8日 | Rui | 修改文档的编写方式 | 使用 Jupyter notebook 进行编写文档 |
| 2022年1月18日 | Rui | 修改文档，增加效果图 | 通过测试的平台有 MaixII-Dock、树莓派、 maixsense，使用的是 MaixPy3 0.4.0 |
| 2022年3月18日 | Rui | 增加 GPIO 模式设置方式，添加 MaixSense  | --- |
| 2022年3月25日 | Rui | 修改GPIO的一些使用方式和主要事项 | --- |

## 简介

GPIO, 全称 General-Purpose Input/Output（通用输入输出），是一种软件运行期间能够动态配置和控制的通用引脚。所有的 GPIO 在上电后的初始状态都是输入模式，可以控制电平的输出和获取外设的电平变化。

## 在？点个灯？
点灯就是通过控制 GPIO 进行高低电平的输出，使用之前需要加载对应板子的 GPIO 配置。通过查看开发板管脚定义图，确定所使用的 IO 管脚号，例如下面使用的是 MaixII-Dock 的 GPIO PH6 管脚，则需要使用 gpio.gpio(6, "H", 1)，进行定义，完整代码如下：

In [1]:
import time
from maix import gpio
led = gpio.gpio(6, "H", 1)
print(led.source)
while True:
    led.set_value(0)
    print(led.get_value())
    time.sleep(0.5)
    led.set_value(1)
    print(led.get_value())
    time.sleep(0.5)
led.release()

[ rpyc-kernel ]( running at Wed Jan  5 17:30:51 2022 )
GPIO chip 1 bank H line 6
0
1
0
1
0
1



接上一个 LED 灯，运行效果如下

![GPIO](./../asserts/GPIO.gif)

> 没有 LED 灯的，可以通过直接操作 GPIO PH14 来控制板载LED

<img src="./../asserts/M2Dock_pin.jpg" width=450px>
<img src="./../asserts/R329-pin.jpg" height=450px>

> 目前 MaixSense 只有 PH 组的可以使用，后续会修复剩下的 GPIO。（2022.04.11）

## 弄个按键

将 GPIO 设置成输入模式，才能获取板子的电平变化。需要在 GPIO 实例化的时候添加多一个参数，来设置 GPIO 的模式。

```python
key = gpio.gpio(6, "H", 1, line_mode = 2)
```
> 2 为设置成输入模式，该参数默认为 1，即输出模式

GPIO 使用结束之后需要将其释放，否则会处于占用状态，通过 gpio.release() 释放 GPIO 或者使用一个类封装起来，完整代码如下：

In [1]:
import time
class BUTTON:
    def __init__(self, line, bank, chip, mode):
        from maix import gpio
        self.button = gpio.gpio.gpio(line, bank, chip, mode)
    def is_pressed(self):
        if self.button.get_value() != 1:
            return True
    def __del__(self):
        self.button.release()

global key
key = BUTTON(6, 2)
print(key.button.source)

[ rpyc-kernel ]( running at Fri Mar 25 14:47:36 2022 )
GPIO chip 1 bank H line 6


检测按键是否按下了

In [1]:
while True:
    if key.is_pressed():
        print("pressed!!")

[ rpyc-kernel ]( running at Fri Mar 25 14:47:40 2022 )
pressed!!
pressed!!
pressed!!
pressed!!



Traceback (most recent call last):
  File "<string>", line unknown, in <module>
Remote.KeyboardInterrupt


## 了解更多

Linux 系统中，使用 GPIO 作为电平的输出口，我们需要关注 [GPIO_line 、GPIO_bank 和 GPIO_chip](https://www.baidu.com/s?ie=utf-8&wd=GPIO_line%20%20GPIO_bank%20GPIO_chip)，这个三个参数。以上面的例程为例子来说，使用 MaixII-Dock 上的 GPIO6，通过查看开发板上引出口的丝印，可以知道 GPIO6 绑定在 PH6 的这个管脚上。

通过查看 [V831_PIN 功能介绍](https://thoughts.teambition.com/share/600659e9823a3d004a4e1c7a#slate-title)得知，V831 只能使用 gpiochip1。这时我们知道了三个参数分别为6、H 和 1。 这时可以使用以下的代码进行 GPIO 的实例化

In [1]:
from maix import gpio
import time
led = gpio.gpio(6, "H", 1)
print(led.source)
for i in range(3):
    led.set_value(0)
    print(led.get_value())
    time.sleep(0.5)
    led.set_value(1)
    print(led.get_value())
    time.sleep(0.5)
led.release()

[ rpyc-kernel ]( running at Wed Jan  5 17:31:00 2022 )
GPIO chip 1 bank H line 6
0
1
0
1
0
1




> 如果 gpio_chip 值选择错误了，就会报 `line offset out of range` 错误
> 更多的关于 Linux_gpio 的使用可以查看[大佬鼠](https://www.cnblogs.com/juwan/p/14336100.html)的博客
